2018년 데이터만 가지고 트리모델 만드는 중

In [ ]:
import pickle
import numpy as np
import pandas as pd
import gc


with open('D:/inputs/24/tree_2018.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/tree_y_2018.pickle', 'rb') as f:
    train_y = pickle.load(f)
train_y = train_y.reshape(train_y.shape[0],40, 40, 1)    
with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)
    


In [ ]:
pd.options.display.max_columns = 100

In [ ]:
from custom_metric import *

In [ ]:
train = train.reshape(train.shape[0] * train.shape[1] * train.shape[2], train.shape[3])
test = test.reshape(test.shape[0] * test.shape[1] * test.shape[2], test.shape[3])
train_y = train_y.reshape(train_y.shape[0] * train_y.shape[1] * train_y.shape[2], train_y.shape[3])
train = np.concatenate([train, train_y], axis=-1)

In [ ]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
LEN_TRAIN = len(train)

In [ ]:
all_df = pd.concat([train, test])

In [ ]:
del train
del test

In [ ]:
all_df.columns = ['v1', 'h1', 'v2', 'h2', 'v3', 'v4', 'h4', 'v5', 'h5', 'surface', 'GMI_y', 'GMI_x', 'DPR_y', 'DPR_x', 'target']
all_df['surface'] = all_df['surface'].astype('int16')

In [ ]:
del all_df['GMI_y']
del all_df['GMI_x']
del all_df['DPR_y']
del all_df['DPR_x']

In [ ]:
all_df['surface1'] = all_df['surface'] // 100

In [ ]:
all_df['surface1']  = all_df['surface1'].astype('int8')

In [ ]:
all_df['vh1_1'] = all_df['v1'] * np.cos(np.pi / 4) + all_df['h1'] * np.sin(np.pi / 4)  
all_df['vh1_2'] = all_df['v1'] * np.cos(np.pi / 4) - all_df['h1'] * np.sin(np.pi / 4)  

all_df['vh2_1'] = all_df['v2'] * np.cos(np.pi / 4) + all_df['h2'] * np.sin(np.pi / 4)
all_df['vh2_2'] = all_df['v2'] * np.cos(np.pi / 4) - all_df['h2'] * np.sin(np.pi / 4)

all_df['vh4_1'] = all_df['v4'] * np.cos(np.pi / 4) + all_df['h4'] * np.sin(np.pi / 4)
all_df['vh4_2'] = all_df['v4'] * np.cos(np.pi / 4) - all_df['h4'] * np.sin(np.pi / 4)

all_df['vh5_1'] = all_df['v5'] * np.cos(np.pi / 6) + all_df['h5'] * np.sin(np.pi / 6)
all_df['vh5_2'] = all_df['v5'] * np.cos(np.pi / 6) - all_df['h5'] * np.sin(np.pi / 6)

In [ ]:
all_df.columns

In [ ]:
for i in ['v1', 'h1', 'v2', 'h2', 'v3', 'v4', 'h4', 'v5', 'h5', 'vh1_1', 'vh1_2', 'vh2_1', 'vh2_2', 'vh4_1', 'vh4_2', 'vh5_1', 'vh5_2']:
    all_df[i] = round(all_df[i], 0)

In [ ]:
surf1 = all_df.groupby('surface1')['target'].mean().rename('surf1_mean')
all_df = all_df.merge(surf1, how='left', on='surface1')
del surf1

In [ ]:
surf = all_df.groupby('surface')['target'].mean().rename('surf_mean')
all_df = all_df.merge(surf, how='left', on='surface')
del surf

In [ ]:
surf1 = all_df.groupby('surface1')['target'].std().rename('surf1_std')
all_df = all_df.merge(surf1, how='left', on='surface1')
del surf1

In [ ]:
surf = all_df.groupby('surface')['target'].std().rename('surf_std')
all_df = all_df.merge(surf, how='left', on='surface')
del surf

In [ ]:
all_df.head()

### 데이터셋 나누기

In [ ]:
del all_df['target']
train = all_df[:LEN_TRAIN]
test = all_df[LEN_TRAIN:]

In [ ]:
del all_df

In [ ]:
train_y = pd.DataFrame(train_y)

In [ ]:
from sklearn.model_selection import train_test_split
train, val, train_y, val_y = train_test_split(train, train_y, test_size=0.05, random_state=30)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler

# ss = StandardScaler()
# # rs = RobustScaler()
# train = ss.fit_transform(train)

# del ss

In [ ]:
# import pickle

# with open('inputs/train_ss.pickle', 'wb') as f:
#     pickle.dump(train, f, protocol=4)
    
# with open('inputs/train_ss.pickle', 'rb') as f:
#     train = pickle.load(f)
    
# with open('D:/inputs/24/train_y.pickle', 'rb') as f:
#     train_y = pickle.load(f)
# train_y = train_y.reshape(75957, 40, 40, 1)    
# train_y = train_y.reshape(train_y.shape[0] * train_y.shape[1] * train_y.shape[2], train_y.shape[3])

### RandomForest Regressor 모델

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(criterion='mae', verbose=1, n_jobs=-1, max_depth=5)
# rf.fit(train, target)
# print(mae_over_fscore(target, rf.predict(train)))

### Lidge 모델

In [ ]:
# from sklearn.linear_model import RidgeCV, Ridge, Lasso
# from sklearn.metrics import mean_absolute_error

# for alpha in [1, 10, 50, 100, 200, 300]:
#     ridge = Ridge(alpha=alpha)
#     ridge.fit(train, target)
#     print(alpha)
#     print(mae_over_fscore(target, ridge.predict(train)))
#     print('\n')

### Lasso 모델 

In [ ]:
# for alpha in [1, 10, 100]:
#     lasso = Lasso()
#     lasso.fit(train, target)
#     print(mae_over_fscore(target, lasso.predict(train)))

### Catboost Regressor 모델

In [ ]:
from catboost import CatBoostRegressor
# category 데이터에 특히 잘 적용되고,
# 컬럼 랜덤하게 뽑고, row도 랜덤하게 뽑아서 랜덤포레스트와 비슷한 느낌. 파라미터 설정에 신경쓰지 않아도 어느 정도점수 잘나오는 모델

cb = CatBoostRegressor(iterations=300, loss_function='MAE', random_seed = 30, task_type="GPU")
cb.fit(train, np.log(train_y+1), eval_set=(val, np.log(val_y+1)), early_stopping_rounds=10)

In [ ]:
print(mae_over_fscore(train_y, np.exp(cb.predict(train)) - 1 ))

In [ ]:
print(mae_over_fscore(val_y, np.exp(cb.predict(val)) - 1 ))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
pd.DataFrame(cb.get_feature_importance(), index = train.columns)

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.fit(train, train_y, init_model=cb)
print(mae_over_fscore(train_y, cb.predict(train)))

In [ ]:
cb.save_model("cb_model_40",
           format="cbm",
           export_parameters=None,
           pool=None)

In [ ]:
# catboost model load - continuous fit 가능
# cb = CatBoostRegressor(iterations=10, loss_function='MAE')
# cb.load_model("cb_model_40")
# cb.fit(train, target, init_model=cb1)

In [ ]:
print(mae_over_fscore(target, cb1.predict(train)))

10 - 3.8450220123151535  
20 - 3.169688157233239  
30 - 2.930901411484702  
40 - 2.7957138063731715

### lightgbm 모델

In [ ]:
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(device='cpu', n_estimators = 400, early_stopping_rounds=10)
lgb.fit(train, train_y, eval_set=(val, val_y))

In [ ]:
print(mae_over_fscore(train_y, lgb.predict(train)))

In [ ]:
print(mae_over_fscore(val_y, lgb.predict(val)))

- 10 - 5.615421879934444
- 20 - 2.5261612717784985
- 30 - 2.490089590398271
- 40 - 2.478701832641337

아이디어
- temp0~8를 pca로 축소하기